In [1]:
import sys
sys.path.append("../ner/BERT-NER/")

In [2]:
from bert import Ner

model = Ner("../ner/BERT-NER/out/")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
output = model.predict("When did the House of Burgundy revolt against Philip II?")

In [4]:
output

{'When': {'tag': 'O', 'confidence': 0.9998822212219238},
 'did': {'tag': 'O', 'confidence': 0.999711811542511},
 'the': {'tag': 'O', 'confidence': 0.9998853206634521},
 'House': {'tag': 'B-ORG', 'confidence': 0.9997528195381165},
 'of': {'tag': 'I-ORG', 'confidence': 0.9949380159378052},
 'Burgundy': {'tag': 'I-ORG', 'confidence': 0.9900270104408264},
 'revolt': {'tag': 'O', 'confidence': 0.9858018755912781},
 'against': {'tag': 'O', 'confidence': 0.9998939037322998},
 'Philip': {'tag': 'B-PER', 'confidence': 0.9998753070831299},
 'II': {'tag': 'O', 'confidence': 0.9446985721588135},
 '?': {'tag': 'O', 'confidence': 0.36162233352661133}}

In [ ]:
def get_ents(sent):
    output =  model.predict(sent)
    ents = []
    ent = None
    for k, v in output.items():
        assert k is not None, k
        if v['tag'] == 'O' and ent is not None:
            ents.append(ent.lower())
            ent = None
        if v['tag'].startswith('B'):
            if ent is not None:
                ents.append(ent.lower())
            ent = k
        if v['tag'].startswith('I'):
            if ent is None:
                ent = k
            elif k.startswith("'"):
                ent += k
            else:
                ent += ' ' + k
    return ents

In [ ]:
get_ents("When did the House of Burgundy revolt against Philip II?")

In [5]:
import json
import pickle
import random

In [6]:
questions = []
contexts = []
examples = []
labels = []
id2idx = {}
files = ["dataset/train-v2.0.json",  "dataset/dev-v2.0.json"]
for file in files:
    with open(file, 'r') as handle:
        jdata = json.load(handle)
        data = jdata['data']
    for i in range(len(data)):
        section = data[i]['paragraphs']
        for sec in section:
            context = sec['context']
            contexts.append(context)
            qas = sec['qas']
            for j in range(len(qas)):
                question = qas[j]['question']
                is_imp = qas[j]['is_impossible']
                qid = qas[j]['id']
                questions.append(question)
                labels.append(is_imp)
                examples.append((len(contexts)-1, len(questions)-1))
                id2idx[qid] = len(questions)-1

In [18]:
sampled_examples_ans = []
sampled_examples_unans = []
for e in random.sample(examples, len(examples)):
    if labels[e[1]] == True and len(sampled_examples_unans) < 50:
        sampled_examples_unans.append(e)
    elif labels[e[1]] == False and len(sampled_examples_ans) < 50:
        sampled_examples_ans.append(e)
       

In [19]:
def get_ents(sent):
    output =  model.predict(sent)
    ents = {}
    ent = None
    for k, v in output.items():
        assert k is not None, k
        if v['tag'] != 'O':
            ents[k.lower()] = v['tag']
    return ents

In [20]:
ent_not_mentioned_ans = 0
ent_not_mentioned_unans = 0

for ci, qi in sampled_examples_ans:
    q = questions[qi]
    c = contexts[ci]
    cents = get_ents(c)
    ents = get_ents(q)
    for ent in ents.keys():
        if ent not in cents.keys():
            ent_not_mentioned_ans += 1
            print("ANS", ents)
            print(cents)
            print(q)
            print(c)
            print("#"*20)
    

for ci, qi in sampled_examples_unans:
    q = questions[qi]
    c = contexts[ci]
    ents = get_ents(q)
    cents = get_ents(c)
    for ent in ents.keys():
        if ent not in cents.keys():
            ent_not_mentioned_unans += 1
            print("UNANS", ents)
            print(cents)
            print(q)
            print(c)
            print("#"*20)

print(ent_not_mentioned_ans, ent_not_mentioned_unans)

ANS {'crucifixion': 'B-MISC'}
{'calvary': 'I-LOC', 'english': 'B-MISC', 'latin': 'B-MISC', 'vulgate': 'B-MISC', 'gospels': 'B-MISC', 'aramaic': 'B-MISC', 'gûlgaltâ': 'B-LOC', 'jesus': 'B-PER', 'jewish': 'B-MISC', 'roman': 'B-MISC'}
Where was the location of the Crucifixion detailed?
Calvary as an English name for the place is derived from the Latin word for skull (calvaria), which is used in the Vulgate translation of "place of a skull", the explanation given in all four Gospels of the Aramaic word Gûlgaltâ which was the name of the place where Jesus was crucified. The text does not indicate why it was so designated, but several theories have been put forward. One is that as a place of public execution, Calvary may have been strewn with the skulls of abandoned victims (which would be contrary to Jewish burial traditions, but not Roman). Another is that Calvary is named after a nearby cemetery (which is consistent with both of the proposed modern sites). A third is that the name was der

UNANS {'latin': 'B-MISC', 'rjecnik': 'B-MISC'}
{'croatian': 'B-MISC', 'latin': 'B-MISC', 'gajica': 'B-MISC', 'ljudevit': 'B-PER', 'gaj': 'I-PER', 'czech': 'B-MISC', 'polish': 'B-MISC', 'rječnik': 'B-ORG', 'yugoslav': 'B-ORG', 'academy': 'I-ORG', 'sciences': 'I-ORG', 'arts': 'I-ORG', 'zagreb': 'B-LOC', 'serbo-croatian': 'B-MISC', 'slavic': 'B-MISC', 'cyrillic': 'B-MISC'}
What is standard Latin represented as in the Rjecnik hrvatskog ili srpskog jezika?
The Croatian Latin alphabet (Gajica) followed suit shortly afterwards, when Ljudevit Gaj defined it as standard Latin with five extra letters that had diacritics, apparently borrowing much from Czech, but also from Polish, and inventing the unique digraphs "lj", "nj" and "dž". These digraphs are represented as "ļ, ń and ǵ" respectively in the "Rječnik hrvatskog ili srpskog jezika", published by the former Yugoslav Academy of Sciences and Arts in Zagreb. The latter digraphs, however, are unused in the literary standard of the language. All

5 12


# Mask sentences

In [37]:
raw_targets = {}
for i in range(24):
    with open("qparts/copy_parts2/parsed_qs_labels%s.pkl"%str(i), "rb") as handle:
        raw_targets.update(pickle.load(handle))

In [43]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
import spacy

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [77]:
def x_in_y_int(query, base):
    try:
        l = len(query)
    except TypeError:
        l = 1
        query = type(base)((query,))

    for i in range(len(base)):
        if base[i:i + l] == query:
            return i
    return -1


def mask_question(tokens_b, target_span):
    span_start = x_in_y_int(target_span, tokens_b)
    print(span_start)
    if span_start >= 0:
        for i in range(span_start, span_start+len(target_span)):
            del tokens_b[span_start]
        tokens_b.insert(span_start, '[MASK]')
    else:
        # find first tok
        first_tok = tokens_b.index(target_span[0])
        for tok in tokens_b:
            if tok in target_span:
                tokens_b.remove(tok)
        tokens_b.insert(first_tok, '[MASK]')
    return tokens_b

In [73]:
for i in random.sample(raw_targets.keys(),1 ):
    targs = [word for word, _ in raw_targets[i]]
    idx = id2idx[i]
    question = questions[idx]
    print(question)
    targ = random.sample(targs, 1)[0]
    print(targ)
    print()
    print(mask_question(tokenizer.tokenize(question), targ))

In what borough did the Stonewall riots happen?
['in', 'what', 'borough']

['[MASK]', 'what', 'did', 'the', 'stone', '##wall', 'riots', 'happen', '?']


In [78]:

print(question)
print(targ)
print()
print(mask_question(tokenizer.tokenize(question), targ))

In what borough did the Stonewall riots happen?
['in', 'what', 'borough']

0
['[MASK]', 'did', 'the', 'stone', '##wall', 'riots', 'happen', '?']
